<a href="https://colab.research.google.com/github/tinggia/bda_stockPrediction/blob/main/202303.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
pip install monpa

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import pandas as pd
import numpy as np
import datetime
import warnings
warnings.filterwarnings("ignore")
import monpa
from monpa import utils
import re
# 這個function用來將字串以正則化處理去除中文字元以外的字元
def clearSentence(sentence):
    return re.sub(r'[^\u4e00-\u9fa5]+', '', sentence)

# 我們從stopwords_zh.txt這個檔案中匯入繁體中文的停用詞
file = open('/stopwords_zh.txt','r')
stopwords = file.read().splitlines() 
file.close()

+---------------------------------------------------------------------+
  Welcome to MONPA: Multi-Objective NER POS Annotator for Chinese
+---------------------------------------------------------------------+
已找到 model檔。Found model file.


In [14]:
news = pd.read_excel("/news.xls")
# 設定訓練資料集的開始日期與結束日期
train_startDate = datetime.date(2022,12,1)
train_endDate = datetime.date(2023,2,28)

In [15]:
# 透過monpa對文章進行斷詞處理，並將個別斷詞結果寫在tokenStr這個字串變數中，最後再將訓練集中所有字串存在tokenStr_list中
train_tokenStr_list = []
news['post_time'] = pd.to_datetime(news['post_time']).dt.date
for i in list(news[news['post_time'].between(train_startDate, train_endDate)].index):
    try:
        sentence_list = utils.short_sentence(news['content'][i])
        tokenStr = str()
        for sentence in sentence_list:
            sentence = clearSentence(sentence)
            tokens = monpa.cut(sentence)
            tokenStr += ' '.join(tokens)
        train_tokenStr_list.append(tokenStr)
    except:
        train_tokenStr_list.append('')

In [13]:
print(train_tokenStr_list)

['隨著 手機 市場 進入 傳統 淡季 先前 手機 面板 價格 在 經過 先前 跌幅 收斂 之後 年月 跌幅 又 有 擴大 的 跡象 表示 年 全年 手機 面板 價格 平均 跌幅 高達 預期 月 面板 價格 跌幅 約 美元 剛性 面板 價格 續跌 美元 柔性 面板 價格 估 跌 美元儘管 在 年月 迎來 了 大陸 雙 十一 歐美 黑色 星期 五 等 促銷 活動 但 受 疫情 和 全球 經濟 持續 低迷 的 影響 市場 消費 熱度 不及 往年 全球 智慧型 手機 市場 未 能 扭轉 下滑 態勢 在 終端 需求 低迷 的 情況 下 上游 面板 需求 暫時 沒有 反轉 向 上 的 預期 手機 面板 價格 下滑 貫穿年 全 年 全年 價格 平均 跌幅 高達指出 隨著 韓系 面板 廠商 陸續 退出 面板 市場 待 大 尺寸 面板 價格 企穩 後高 世代線 的 經營 壓力 減小 利用 手機 產品 低價 搶占 市場 以 填補 產能 的 意願 將 明顯 減弱 但是 短期 內 手機 面板 價格 仍 將 承壓 手機 面板 價格 自 年月 進入 下行 週期年 價格 全年 持續 處於 下跌 的 走勢 而且 在 第四 季 降幅 擴大整體 來 看年 面板 價格 累計 降幅 面板 累計 降幅 面板廠 產能 持續 轉向 工控車 載 等 應用 產品 主流 面板廠 持續 加大 白牌 市場 出貨 年 柔性 手機 面板 價格 累計 降幅 高達 剛性 手機 面板 價格 累計 降幅 主力 面板 供應商 三星 顯示器 出貨量 及 市場 佔有率 均 明顯 下滑 尤其 在 中國 市場 出貨量 銳減預測 在 需求 沒有 明顯 好轉 因素 的 情況 下 隨著 市場 進入 傳統 淡季 智慧型 手機 面板 價格 將 持續 走低 年月 和 年月 面板 價格 每 月 將 下降 美元 剛性 面板 價格 將 繼續 下降 美元 柔性 面板 價格 預估 約 下降 美元', '日期 年月日 公司 名稱 亞帝歐 主旨 亞帝歐 光電 科技 吳江有限公司 處 份 銀行組 合式 商品 發言人 黃正心 說明 標的 物 之 名稱 及 性質 屬 特別 股 者 並 應 標明 特別股 約定 發行 條件如 股息率 等 保本 投資型 組合式 商品 事實 發生 日 交易 數量 每 單位 價格 及 交易 總金額 不 適用 不 適用 人民幣 元交易 相對 人 及 

In [16]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
vectorizer = TfidfVectorizer(stop_words=stopwords)
X_train = vectorizer.fit_transform(train_tokenStr_list)
X_train = pd.DataFrame(X_train.toarray(),columns=vectorizer.get_feature_names_out())
X_train
y_train = news[news['post_time'].between(train_startDate, train_endDate)]['label']

chi2_selector = SelectKBest(chi2, k = 10000)
chi2_selector.fit(X_train, y_train)
kbest_vocabs = X_train.columns[chi2_selector.get_support()]
X_train = X_train[kbest_vocabs]
X_train

,一一一年度,一上,一二,一二季,一兆,一共,一口氣,一城,一姐,一字,...,齊漲,齊發,齊跌三,龍潭,龍邦,龍邦國際,龍門,龍頭股台積電,龐大,龔明鑫
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2013,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2014,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2015,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2016,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [17]:
# 首先先計算4月份的文章個別的tfidf
test_startDate = datetime.date(2021,4,1)
test_endDate = datetime.date(2021,4,30)

test_tokenStr_list = []
for i in list(news[news['post_time'].between(test_startDate, test_endDate)].index):
    try:
        txt = clearSentence(news['content'][i])
        sentence_list = utils.short_sentence(txt)
        tokenStr = str()
        for sentence in sentence_list:
            tokens = monpa.cut(sentence)
            tokenStr += ' '.join(tokens)
        test_tokenStr_list.append(tokenStr)
    except:
        test_tokenStr_list.append('')
y_test =news[news['post_time'].between(test_startDate, test_endDate)]['label']
vectorizer = TfidfVectorizer(stop_words=stopwords)
X_test = vectorizer.fit_transform(test_tokenStr_list)
X_test = pd.DataFrame(X_test.toarray(),columns=vectorizer.get_feature_names_out())
X_test = X_test.reindex(kbest_vocabs, axis=1, fill_value=0)
X_test

,一一一年度,一上,一二,一二季,一兆,一共,一口氣,一城,一姐,一字,...,齊漲,齊發,齊跌三,龍潭,龍邦,龍邦國際,龍門,龍頭股台積電,龐大,龔明鑫
0,0,0,0.0,0,0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0,0,0,0,0,0,0.000000,0
1,0,0,0.0,0,0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0,0,0,0,0,0,0.000000,0
2,0,0,0.0,0,0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0,0,0,0,0,0,0.000000,0
3,0,0,0.0,0,0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0,0,0,0,0,0,0.039063,0
4,0,0,0.0,0,0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0,0,0,0,0,0,0.000000,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
568,0,0,0.0,0,0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0,0,0,0,0,0,0.000000,0
569,0,0,0.0,0,0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0,0,0,0,0,0,0.000000,0
570,0,0,0.0,0,0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0,0,0,0,0,0,0.000000,0
571,0,0,0.0,0,0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0,0,0,0,0,0,0.000000,0


In [18]:
#用svm
from sklearn import metrics
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
SVM_model = SVC(kernel='rbf', C=1.0)
x_train, x_test, ytrain, ytest = train_test_split(X_train, y_train, test_size = 0.1, stratify=y_train)
SVM_model.fit(x_train, ytrain)

SVC()

In [19]:
predicted_results = []
expected_results = []

expected_results.extend(y_test)

print(SVM_model.predict(X_test))
predicted_results.extend(SVM_model.predict(X_test))
print(metrics.accuracy_score(expected_results, predicted_results))
print(metrics.confusion_matrix(expected_results, predicted_results, labels=[ '漲', '跌']))

['跌' '跌' '跌' '漲' '跌' '跌' '跌' '跌' '跌' '漲' '漲' '跌' '跌' '跌' '跌' '跌' '跌' '跌'
 '跌' '漲' '跌' '跌' '跌' '漲' '漲' '漲' '漲' '跌' '跌' '跌' '跌' '跌' '跌' '跌' '跌' '跌'
 '跌' '跌' '跌' '跌' '跌' '漲' '跌' '跌' '跌' '跌' '跌' '跌' '跌' '漲' '漲' '跌' '跌' '跌'
 '跌' '跌' '跌' '跌' '跌' '跌' '跌' '跌' '跌' '漲' '跌' '跌' '漲' '跌' '跌' '跌' '跌' '跌'
 '跌' '跌' '跌' '跌' '跌' '跌' '跌' '跌' '跌' '跌' '跌' '跌' '跌' '跌' '跌' '跌' '跌' '跌'
 '跌' '跌' '跌' '跌' '跌' '跌' '跌' '跌' '跌' '漲' '漲' '跌' '跌' '漲' '跌' '跌' '跌' '跌'
 '跌' '跌' '跌' '跌' '跌' '跌' '跌' '跌' '跌' '跌' '跌' '跌' '跌' '跌' '跌' '跌' '跌' '跌'
 '跌' '跌' '跌' '跌' '跌' '跌' '跌' '跌' '跌' '跌' '跌' '跌' '跌' '跌' '跌' '跌' '跌' '跌'
 '跌' '跌' '漲' '跌' '跌' '跌' '跌' '跌' '漲' '跌' '跌' '跌' '跌' '跌' '跌' '跌' '跌' '跌'
 '跌' '漲' '跌' '漲' '跌' '跌' '跌' '跌' '跌' '跌' '漲' '漲' '漲' '跌' '跌' '跌' '跌' '跌'
 '跌' '跌' '跌' '跌' '跌' '跌' '跌' '跌' '跌' '跌' '跌' '跌' '跌' '跌' '跌' '漲' '跌' '跌'
 '跌' '跌' '跌' '跌' '跌' '跌' '漲' '跌' '跌' '跌' '跌' '漲' '跌' '跌' '漲' '跌' '漲' '跌'
 '跌' '跌' '跌' '跌' '跌' '跌' '跌' '跌' '跌' '跌' '跌' '跌' '跌' '跌' '跌' '漲' '跌' '跌'
 '跌' '跌' '跌' '跌' '跌' '跌' '跌' '跌' '漲' '跌' '跌' '跌' '跌